In [7]:
from math import exp

u, v = 0, 0
for i in range(5):
    uvalue = exp(u) + v * exp(u * v) + 2 * u - 2 * v - 3
    vvalue = 2 * exp(2 * v) + u * exp(u * v) - 2 * u + 4 * v - 2
    u, v = u - 0.01 * uvalue, v - 0.01 * vvalue

# print (u)
# print (v)
E = exp(u) + exp(2 * v) + exp(u * v) + pow(u, 2) - 2 * u * v + 2 * pow(v, 2) - 3 * u - 2 * v
print (E)


2.8250003566832635


In [27]:
from math import exp
from numpy import *
u, v = 0, 0
for i in range(5):
    H = mat([[exp(u) + pow(v, 2) * exp(u * v) + 2, exp(u * v) + u * v * exp(u * v) - 2],[exp(u * v) + u * v * exp(u * v) - 2, 4 * exp(2 * v) + pow(u, 2) * exp(u * v) + 4]])
    inverse = H.I
    descent = [exp(u) + v * exp(u * v) + 2 * u - 2 * v - 3, 2 * exp(2 * v) + u * exp(u * v) - 2 * u + 4 * v - 2]
    matrix = [u, v] - inner(inverse, descent)
    u = matrix[0,0]
    v = matrix[0,1]

E = exp(u) + exp(2 * v) + exp(u * v) + pow(u, 2) - 2 * u * v + 2 * pow(v, 2) - 3 * u - 2 * v
print (E)


2.36082334564


In [211]:
# question 13~15
# linear regression
from random import uniform
from numpy import *

def sign(x):
    if x > 0 : return 1
    else: return -1
    
def noise():
    x = random.uniform(0,1)
    if x > 0.1:
        return 1
    else :
        return -1
    
def generate_X():
    X = []
    for i in range(DATA_NUMBER):
        x0 = 1
        x1 = random.uniform(-1, 1)
        x2 = random.uniform(-1, 1)
        X.append([x0, x1, x2])
    return X

def generate_Y(X):
    Y = []
    for x in X:
        y = sign(pow(x[1], 2) + pow(x[2], 2) - 0.6) * noise()
        Y.append(y)
    return Y

def feature_transform(X):
    Z = []
    for x in X:
        # 1, x1, x2, x1x2, x1 ^2, x2 ^2
        w0 = 1
        w1 = x[1]
        w2 = x[2]
        w3 = x[1] * x[2]
        w4 = power(x[1], 2)
        w5 = power(x[2], 2)
        Z.append([w0,w1,w2,w3,w4,w5])
    return Z

def get_W(X, Y):
    W = linalg.inv(transpose(X).dot(array(X))).dot(transpose(X)).dot(Y)
    return W
    
def test(X, Y, W):
    train_Y = array(W).dot(transpose(X))
    count = 0
    for i in range(len(Y)):
        if sign(train_Y[i]) != Y[i]:
            count += 1
    return count / len(Y)

DATA_NUMBER = 1000
LOOP_TIMES = 1000

def main():
    sum_W = [0,0,0,0,0,0]
    Ein_sum = 0
    for i in range(LOOP_TIMES):
        X = generate_X()
        Y = generate_Y(X) # add noise
        # question 14
        Z = feature_transform(X)
        W = get_W(Z, Y)
        sum_W = sum_W + W
        Ein_temp = test(Z, Y, W)
        Ein_sum += Ein_temp
    Ein_aver = Ein_sum / LOOP_TIMES
    aver_W = sum_W / LOOP_TIMES
    
    #question 15
    Eout_sum = 0
    for i in range(LOOP_TIMES):
        test_X = generate_X()
        test_Y = generate_Y(test_X)
        test_Z = feature_transform(test_X)
        Eout_sum += test(test_Z, test_Y, aver_W)
    Eout_aver = Eout_sum / LOOP_TIMES
    print (Ein_aver)
    print(Eout_aver)
    # print(aver_W)
    

if __name__ == '__main__':
    main()

0.1234499999999999
0.12287999999999985


In [278]:
# question 18~20
# Logistic Regression
import numpy as np
import random

def sign(x):
    if x > 0 : return 1
    else : return -1

def read_data(input_file):
    f = open(input_file)
    X = []
    Y = []
    for line in f:
        strs = line.split()
        x = [1] + [float(v) for v in strs[:-1]]
        y = int(strs[-1])
        X.append(x)
        Y.append(y)
    return X, Y

def sigmoid(s):
    return 1 / (1 + exp(-s))
    
def calculate_gratitude_Ein(X, Y, W):
    sum = [0 for i in range(len(X[0]))]
    for i in range(len(Y)):
        sum += sigmoid(-Y[i] * array(W).dot(np.transpose(X[i]))) * (-Y[i] * array(X[i]))
    return sum / len(Y)
    
def batchGradient(X, Y):
    W = [0 for i in range(len(X[0]))]
    for k in range(LOOP_TIMES):
        for i in range(len(Y)):
            if (sign(array(W).dot(np.transpose(X[i]))) != Y[i]):
                gratitude_Ein = calculate_gratitude_Ein(X, Y, W)
                W = W - eta * gratitude_Ein
    return W

def stochasticGradient(X, Y):
    W = [0 for i in range(len(X[0]))]
    for k in range(LOOP_TIMES):
        i = random.randrange(len(Y))
        W = W + eta * sigmoid(- Y[i] * np.array(W).dot(np.transpose(np.array(X[i])))) * Y[i] * np.array(X[i])
    return W
            
def test(X, Y, W):
    count = 0
    for i in range(len(Y)):
        if sign(array(W).dot(np.transpose(X[i]))) != Y[i]:
            count += 1
    return count / len(Y)

TRAIN_DATA = 'train_data.dat'
TEST_DATA = 'test_data.dat'
LOOP_TIMES = 40000
eta = 0.01

def main():
    train_X, train_Y = read_data(TRAIN_DATA)
    # question 18
    # W = batchGradient(train_X, train_Y)
    # question 19
    W = stochasticGradient(train_X, train_Y)
    Ein = test(train_X, train_Y, W)
    test_X, test_Y = read_data(TEST_DATA)
    Eout = test(test_X, test_Y, W)
    # print(Ein)
    # print(Eout)
    
if __name__ == '__main__':
    main()